In [23]:
import pandas as pd
df=pd.read_csv('04_02_2019.csv')

#data replacing
rename_stages = {
    'THORAIPAKK|THURAIPAKK': 'THORAIPAKKAM',
    'ANNA UNIVE': 'ANNA UNIV',
    'MOOTAIKARA': 'M K CHAVADI',
    'SHOZHANGAN|SHOZHINGAN': 'SHOLINGANALLUR',
    'KELAMBAKKA':'KELAMBAKKAM',
    'KOMAN NGR': 'KUMARAN NG',
    'M.K. CHAVA': 'M K CHAVADI',
    'WOMENS POL': 'WPTC',
    'THIRUPPORU|TIRUPPORUR': 'THIRUPORUR',
}
df = df.replace(rename_stages, regex=True)

In [24]:
df['Source'] = df['Source'].str.strip()
df['From Stage'] = df['From Stage'].str.strip()
df['To Stage'] = df['To Stage'].str.strip()
cols=['Schedule Name','Adult','From Stage','To Stage','Trip Start Time','Trip End Time','Source','Destination']
df=df[cols]

name=input("Enter Schedule Name: ")
source=input("Enter Source: ")
destination=input("Enter Destination: ")
start_time=input("Enter Trip Start Time ")
end_time=input("Enter Trip End Time ")

df=df[df['Schedule Name']==name]
df=df[(df['Source'] == source) & (df['Destination']==destination)]
df=df[(df['Trip Start Time']==start_time) & (df['Trip End Time']==end_time)]

cols=['From Stage','To Stage','Adult']
df=df[cols]
df

Enter Schedule Name: 519-A1-AS-MDE
Enter Source: T.NAGAR
Enter Destination: THIRUPORUR
Enter Trip Start Time 08:30:00
Enter Trip End Time 10:46:00


,From Stage,To Stage,Adult
48903,T.NAGAR,ANNA UNIV,1
48904,T.NAGAR,KANDANCHAV,1
48905,T.NAGAR,SRP TOOLS,1
48906,T.NAGAR,NAVALUR,1
48907,T.NAGAR,M K CHAVADI,1
...,...,...,...
50385,CHEMMANCHE,NAVALUR,1
50389,CHEMMANCHE,KELAMBAKKAM,1
50438,NAVALUR,KELAMBAKKAM,1
50440,NAVALUR,KELAMBAKKAM,1


In [25]:
#displaying from_stages 
arr1=df['From Stage'].unique()
arr1

array(['T.NAGAR', 'SAIDAPET', 'ANNA UNIV', 'KUMARAN NG', 'CHURCH',
       'PAL. CHEMI', 'WPTC', 'SRP TOOLS', 'THORAIPAKKAM',
       'SHOLINGANALLUR', 'CHEMMANCHE', 'NAVALUR', 'HINDUSTAN'],
      dtype=object)

In [26]:
arr2=df['To Stage'].unique()
arr2

array(['ANNA UNIV', 'KANDANCHAV', 'SRP TOOLS', 'NAVALUR', 'M K CHAVADI',
       'SHOLINGANALLUR', 'KARAPAKKAM', 'THORAIPAKKAM', 'SAIDAPET',
       'THIRUPORUR', 'KELAMBAKKAM', 'CHURCH', 'KUMARAN NG', 'WPTC',
       'PAL. CHEMI', 'HINDUSTAN'], dtype=object)

In [27]:
bus_stages = [
    'T.NAGAR', 'SAIDAPET', 'ANNA UNIV', 'WPTC', 'SRP TOOLS',
    'KANDANCHAV', 'THORAIPAKKAM', 'M K CHAVADI', 'KARAPAKKAM',
    'SHOLINGANALLUR', 'KUMARAN NG', 'CHEMMANCHE', 'NAVALUR',
    'SIPCOT', 'CHURCH', 'PAL. CHEMI', 'HINDUSTAN', 'KELAMBAKKAM',
    'KOMAN NAGAR', 'ENGG', 'CHENGAMMAL', 'KALAVAKKAM','THIRUPORUR'
]

stage_mapping = {stage: i for i, stage in enumerate(bus_stages)}

df['From Stage'] = df['From Stage'].map(stage_mapping)
df['To Stage'] = df['To Stage'].map(stage_mapping)

df.dropna(subset=['From Stage', 'To Stage'], inplace=True)

#create an empty OD matrix
od_matrix = pd.DataFrame(index=range(len(bus_stages)), columns=range(len(bus_stages))).fillna(0)

# populating the matrix 
for index, row in df.iterrows():
    source = int(row['From Stage'])
    destination = int(row['To Stage'])
    adult_count = row['Adult']
    od_matrix.loc[source, destination] += adult_count if not pd.isna(adult_count) else 0

od_matrix['Boarding'] = od_matrix.sum(axis=1)
od_matrix.loc['ALIGHTING'] = od_matrix.sum(axis=0)

#display the entire df
print(od_matrix.to_string())


           0  1  2  3  4  5  6   7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  Boarding
0          0  1  1  0  8  6  2  10  3  6   1   0   1   0   1   0   0   1   0   0   0   0   1        42
1          0  0  0  1  0  2  0   1  0  0   0   0   0   0   0   0   0   1   0   0   0   0   0         5
2          0  0  0  0  0  0  1   1  1  0   0   0   1   0   2   0   0   2   0   0   0   0   0         8
3          0  0  0  0  0  1  1   1  3  0   3   0   0   0   0   0   0   0   0   0   0   0   0         9
4          0  0  0  0  0  0  0   0  0  0   0   0   0   0   0   0   0   1   0   0   0   0   0         1
5          0  0  0  0  0  0  0   0  0  0   0   0   0   0   0   0   0   0   0   0   0   0   0         0
6          0  0  0  0  0  0  0   0  0  0   0   0   1   0   0   0   0   1   0   0   0   0   0         2
7          0  0  0  0  0  0  0   0  0  0   0   0   0   0   0   0   0   0   0   0   0   0   0         0
8          0  0  0  0  0  0  0   0  0  0   0   0   0   0   0   0   0   0 

In [28]:
#TO DISPLAY IN BUS STAGE NAMES INSTEAD OF NUMBERS
od_matrix = pd.DataFrame(index=bus_stages, columns=bus_stages).fillna(0)

# Populate the matrix 
for index, row in df.iterrows():
    source = bus_stages[int(row['From Stage'])]
    destination = bus_stages[int(row['To Stage'])]
    adult_count = row['Adult']
    od_matrix.loc[source, destination] += adult_count if not pd.isna(adult_count) else 0

od_matrix['Boarding'] = od_matrix.sum(axis=1)
od_matrix.loc['ALIGHTING', :] = od_matrix.sum(axis=0)

od_matrix

,T.NAGAR,SAIDAPET,ANNA UNIV,WPTC,SRP TOOLS,KANDANCHAV,THORAIPAKKAM,M K CHAVADI,KARAPAKKAM,SHOLINGANALLUR,...,CHURCH,PAL. CHEMI,HINDUSTAN,KELAMBAKKAM,KOMAN NAGAR,ENGG,CHENGAMMAL,KALAVAKKAM,THIRUPORUR,Boarding
T.NAGAR,0.0,1.0,1.0,0.0,8.0,6.0,2.0,10.0,3.0,6.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,42.0
SAIDAPET,0.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0
ANNA UNIV,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,8.0
WPTC,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0
SRP TOOLS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
KANDANCHAV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
THORAIPAKKAM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0
M K CHAVADI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
KARAPAKKAM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SHOLINGANALLUR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [29]:
#saving in an excel file 
od_matrix.to_excel('odm_file.xlsx')